# WDC Code Snippet Library

This notebook creates a code snippet library that you can use to quickly add commonly used features and functions to the notebooks you are developing. It needs to be setup before you start using in and populated with the shared code snippets in the library folder.

**To add the snippets from this library, select 'Run' on the top menu, and then select 'Run All Cells'**

In [ ]:
import os
import json
import glob
from datetime import datetime

config_file_path = "/home/jovyan/.jupyter/lab/user-settings/jupyterlab-code-snippets/snippets.jupyterlab-settings"

def extract_id(json):
    try:
        return json['id']
    except KeyError:
        return 0

def build_library():
    snippets = []
    user_snippets = []
    counter = 0
    config = {}
    
    for filepath in glob.glob(os.path.join('library', '*.json')):
        with open(filepath) as f:
            content = json.load(f)
            if not('tags' in content):
                content['tags'] = []
            if not('wdc_library' in content['tags']):
                content['tags'].append('wdc_library')
            snippets.append(content)
            
    snippets.sort(key=extract_id)
        
    # check is the user already has a snippet library and be careful not to obliterate it; instead
    # remove all snippets with 'wdc_library' tag and merge it with the global library of snippets
    if os.path.exists(config_file_path):
        with open(config_file_path) as f:
            content = ""
            while line := f.readline():
                if not line.lstrip().startswith("//"):
                    content = content + line
            try:
                user_config = json.loads(content)
            except:
                print("ERROR: Failed to parse your existing configuration file. Continuing may delete existing snippets so exiting without making changes")
                return
            
        # create a backup
        with open(f'{config_file_path}-backup-{datetime.now().strftime("%Y-%m-%d")}', 'w', encoding='utf-8') as f:
            json.dump(user_config, f, ensure_ascii=False, indent=4)
        
        user_snippets = []
        if "snippets" in user_config:
            for snip in user_config["snippets"]:
                if not("wdc_library" in snip["tags"]):
                    user_snippets.append(snip)
                       
        user_snippets.sort(key=extract_id)
        
    # merge the global list with the user list
    config_snippets = snippets + user_snippets
    for snip in config_snippets:
        snip["id"] = counter
        counter+=1
         
    config['snippets'] = config_snippets
    with open(config_file_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, ensure_ascii=False, indent=4)
     
    print(f"Snippet library updated; {len(snippets)} global snippets and {len(user_snippets)} user snippets")

In [ ]:
build_library()